In [96]:
import os
import random

import torch
from torch.utils.data import Dataset

os.environ["WANDB_DISABLED"] = "true"

model_name = "test-3"
model_type = ""
model_output_dir = f"/workspaces/General/models/{model_name}"
try:
	os.mkdir(model_output_dir)
except FileExistsError:
	pass

In [97]:
class CustomDataset(Dataset):
	_input_id: str = 'input_ids'
	_attention_mask: str = 'attention_mask'

	def __init__(self, text_list, tokenizer, max_length, truncation=False):
		self.input_ids = []
		self.attention_mask = []
		self.labels = []
		for text in text_list:
			encodings_dict = tokenizer(text, truncation=truncation, max_length=max_length)
			self.input_ids.append(torch.tensor(encodings_dict[self._input_id]))
			self.attention_mask.append(torch.tensor(encodings_dict[self._attention_mask]))

	def __len__(self):
		return len(self.input_ids)

	def __getitem__(self, index):
		return self.input_ids[index], self.attention_mask[index]

In [98]:
class QuestionAnswer(object):
	def __init__(self, question: str, answer: str):
		self.question: str = question
		self.answer: str = answer

In [99]:
class DataGenerator(object):
	__bos_token: str = '<|startoftext|>'
	__eos_token: str = '<|endoftext|>'

	# Non-Standard Tokens
	__start_of_question_token: str = '<|startofquestion|>'
	__end_of_question_token: str = '<|endofquestion|>'
	__start_of_reply_token: str = '<|startofreply|>'
	__end_of_reply_token: str = '<|endofreply|>'


	@classmethod
	def create_data_line(cls, data_line: QuestionAnswer) -> str:
		tagged_text: str = cls.__bos_token
		tagged_text += cls.__start_of_question_token
		tagged_text += data_line.question
		tagged_text += cls.__end_of_question_token
		tagged_text += cls.__start_of_reply_token
		tagged_text += data_line.answer
		tagged_text += cls.__end_of_reply_token
		tagged_text += cls.__eos_token

		return tagged_text

	@classmethod
	def get_special_token_dict(cls) -> dict:
		return {
			"bos_token": f"{cls.__bos_token}",
			"eos_token": f"{cls.__eos_token}",
			"additional_special_tokens": [
				f"{cls.__bos_token}",
				f"{cls.__eos_token}",
				f"{cls.__start_of_question_token}"
				f"{cls.__end_of_question_token}"
				f"{cls.__start_of_reply_token}"
				f"{cls.__end_of_reply_token}"
			]}

In [100]:
data_lines = [
	QuestionAnswer(question="what is required to sign up a customer for ENS?", answer="To enroll in the encounter notification service please ensure that your organization has executed a participation agreement with CRISP and updated your organizations Notice of Privacy Practices (NPP). Once those steps are completed the CRISP team will provide a template file for you to securely submit your patient member file to the HIE. Additionally there is a checklist that will allow you to choose what notifications you want to receive and how you want to receive them. All users can use our ENS PROMPT tool at no cost."),
	QuestionAnswer(question="what types of organizations participate in CRISP?", answer="A Health Information Exchange, or HIE, is a way of sharing your health information among participating doctors’ offices, hospitals, care coordinators, labs, radiology centers, and other health care providers through secure, electronic means. The purpose is so that each of your participating healthcare providers can have the benefit of the most recent information available from your other participating providers when taking care of you. When you opt out of participation in the HIE, doctors and nurses will not be able to search for your health information through the HIE to use while treating you. Your physician or other treating providers will still be able to select the HIE as a way to receive your lab results, radiology reports, and other data sent directly to them that they may have previously received by fax, mail, or other electronic communications."),
	QuestionAnswer(question="If I don't want to share my information with CRISP can I choose to opt-out?", answer="Please be advised that opting out does not preclude any participating organization that has received or accessed personal health information via the HIE prior to such opt-out, and incorporated such personal health information into its records, from retaining such information in its records. Additionally, in accordance with the law, Public health reporting, such as the reporting of infectious diseases to public health officials, will still occur through the HIE after you decide to opt out. Controlled Dangerous Substances (CDS) information, as part of the Maryland Prescription Drug Monitoring Program, will continue to be available through the HIE to licensed providers.If you choose to opt out of research only, your information will be available to your treating providers, but will be excluded from any data sets created for researchers."),
	QuestionAnswer(question="what is the value of becoming an affiliate of CRISP Shared Services?", answer="As Maryland’s official regional Health Information Exchange, the CRISP HIE network is comprised of hundreds of connected providers, consisting of hospitals, EMRs, pharmacies, payors, health departments, and health centers."),
	QuestionAnswer(question="what is the CRISP Shared Services?", answer="CRISP Shared Services (CSS) is a non-profit support organization that provides technology infrastructure and other core services to Health Information Exchanges (HIEs) across the US. We are different than a vendor in that each of our Member HIEs participates in the governance of the organization."),
	QuestionAnswer(question="what is the motivation of CRISP Shared Services?", answer="Our primary motivation is to enable and support each local jurisdiction’s Healthcare community so that it can improve health outcomes for its patients. We work with local HIE leadership to implement solutions which best serve the needs of their unique communities, even if those solutions are deployed or built by external vendors."),
	QuestionAnswer(question="what is the mission of CRISP Shared Services?", answer="Our mission is to assist member organizations in achieving economies of scale, pooling innovation efforts, and implementing best practices."),
	QuestionAnswer(question="What is CRISP?", answer="CRISP is the State Designated Health Information Exchange (HIE) for Maryland."),
	QuestionAnswer(question="What does CRISP do?", answer="We are a non-profit organization that facilitates the electronic transfer of clinical information between disparate health information systems."),
	QuestionAnswer(question="Clinical Data", answer="As clinical information is created and shared with CRISP, it is made accessible in near real-time to participating health care providers through the CRISP tools. Providers have the ability to securely look up patient information through the internet. CRISP tools retrieve clinical data from participants and display it in a view-only screen at the point of care."),
	QuestionAnswer(question="Where is CRISP clinical data? When Is It Made Available?", answer="CRISP clinical data is available through the CRISP Portal at no cost to clinical staff. As clinical information is created and shared with CRISP, it is made accessible in real-time to participating healthcare providers across institutional boundaries through the Clinical Information Service. The portal gives providers the ability to securely look up patient information via their browser. It retrieves clinical data from participants and displays it in a view-only screen at the point of care."),
	QuestionAnswer(question="How can I View Patient Data?", answer="The Patient Care Snapshot combines critical information relevant to your role in the patient’s care. It displays data from a variety of sources to provide an at-a-glance view of the patient’s clinical history. Information is presented from a compilation of care management data alongside real-time hospital encounter feeds, up-to-date demographic information, patient to care provider attribution, and clinical summaries of care from our real-time interfaces with providers across the region."),
	QuestionAnswer(question="How can I View Patient Data?", answer="Imaging Worklist allows users to compare images across multiple locations via the Imaging Worklist tab through CRISP’s Portal. Patient images from CRISP participating sites dating back to 01/01/2000 are available through CRISP’s Imaging Worklist as well. In addition, up to four imaging studies can be selected, viewed, and compared at the same time."),
	QuestionAnswer(question="How can I View Patient Data?", answer="Image Exchange is an online image-sharing service that allows CRISP users to view patient diagnostic images in one central location.Currently, 50 hospitals and 9 outpatient groups across Maryland and DC are contributing images to the Image Exchange service. These images are then made available to CRISP users through our portals to facilitate greater collaboration and efficiency among healthcare providers, ultimately leading to higher quality patient care.The diagnostic images are securely stored on servers located within each connected hospital’s local environment. Images taken within the last 90 days are made available to all authorized CRISP users within seconds of collection, while deeper archives of images older than 90 days are available within minutes.CRISP offers two ways of viewing patient clinical data and images. The first option is to directly log on to the CRISP Portal, using specific user credentials. The second option is to access images via the HIE InContext platform within each respective EMR.In addition to the image access provided through Health Records and Imaging Worklist, Image Exchange participants can also request access to the following features: Transfer-to-PACS and Emergent."),
	QuestionAnswer(question="Is access to an Electronic Medical Record (EMR) required to participate in CRISP?", answer="An EMR is not required to access CRISP or the data we provide. CRISP data is accessible through any Internet browser via the CRISP Portal."),
	QuestionAnswer(question="Is there a cost associated with accessing CRISP data?", answer="There is no cost. Access to CRISP is free to ambulatory practices."),
	QuestionAnswer(question="Who do I sign up for CRISP?", answer="To get started, an organization must sign a CRISP Participation agreement for single sites or an agreement for organizations with multiple sites and must update its Notice of Privacy Practices documents. If your organization has already signed a participation agreement, you will be able to skip this step during the onboarding process and simply contact your organization’s CRISP Point of Contact for access.The participation agreement is the uniform data sharing agreement signed by every organization that participates in CRISP. It ensures that everyone sharing data follows the same rules and regulations. Updating your NPP ensures that your practice does its part to inform patients about how CRISP is being used to deliver and coordinate care and informs them of the right to opt-out. Organizations must have CRISP Opt-Out Forms on-site to distribute to patients who ask for one. They must also maintain copies of the Patient Fact sheets in registration areas. New users without an agreement will be prompted to upload one during the onboarding process."),
	QuestionAnswer(question="How do I get started?", answer="In addition, an organization must: 1.) Submit LabCorp/Quest Data Release Form if it wishes to make its results available in CRISP. 2.)Send a patient panel (all patients seen within the last 18 months) using DIRECT secure email to panelupload@crispdirect.org. This patient list will be used to audit your organization’s future activity to help ensure appropriate use. Here is a sample [Patient Panel](https://crisphealth.org/wp-content/uploads/2016/03/Sample-ENS-Patient-Panel.xlsx). 3.)In order to send us your panel, please email [support@crisphealth.org](panelupload@crispdirect.org) to request a DIRECT secure email address. Then, follow these [instructions](https://crisphealth.org/wp-content/uploads/2016/03/How-to-Upload-ENS-Participant-Panel.pdf)"),
	QuestionAnswer(question="Who Contributes Health Information?", answer="All 50 acute-care hospitals in Maryland and 7 acute care hospitals in DC are working with CRISP. To see which of these hospitals are currently providing clinical information, visit our Participating Providers page [here](https://crisphealth.org/about-crisp/connected-providers/).")
] * 100

random.shuffle(data_lines)

In [101]:
import torch
from torch.utils.data import Dataset, random_split

data_generator:DataGenerator = DataGenerator()

generator = torch.Generator()

generator.manual_seed(0)

print(f":: Total Number Of Samples {len(data_lines)}")

train_size = int(0.9 * len(data_lines))
data_lines_processed = []
for elem in data_lines:
	data = data_generator.create_data_line(elem)
	data_lines_processed.append(data)

train_dataset_file, eval_dataset_file = random_split(list(data_lines_processed), [train_size, len(data_lines_processed) - train_size], generator=generator)

train_file = f"{model_output_dir}/train.txt"
eval_file = f"{model_output_dir}/eval.txt"

with open(train_file, 'w', encoding="utf-8") as train_out, open(eval_file, "w", encoding="utf-8") as eval_out:
        train_out.writelines([line + "\n" for line in train_dataset_file])
        eval_out.writelines([line + "\n" for line in eval_dataset_file])

:: Total Number Of Samples 1900


In [102]:
from transformers import GPT2Tokenizer
from transformers.models.gpt2.modeling_gpt2 import GPT2LMHeadModel

data_generator: DataGenerator = DataGenerator()

special_tokens_dict: dict = data_generator.get_special_token_dict()

tokenizer: GPT2Tokenizer = GPT2Tokenizer.from_pretrained(f"gpt2{model_type}")

tokenizer.add_special_tokens(special_tokens_dict)

model: GPT2LMHeadModel = GPT2LMHeadModel.from_pretrained(f"gpt2{model_type}")

model.resize_token_embeddings(len(tokenizer))

model.save_pretrained(model_output_dir)

tokenizer.save_pretrained(model_output_dir)

('/workspaces/General/models/test-3\\tokenizer_config.json',
 '/workspaces/General/models/test-3\\special_tokens_map.json',
 '/workspaces/General/models/test-3\\vocab.json',
 '/workspaces/General/models/test-3\\merges.txt',
 '/workspaces/General/models/test-3\\added_tokens.json')

In [103]:
import json
from simpletransformers.config.model_args import LanguageModelingArgs
from simpletransformers.language_modeling import LanguageModelingModel

torch.cuda.empty_cache()

args = {
    "overwrite_output_dir": True,
    "learning_rate": 1e-4,
    "gradient_accumulation_steps": 100,
    "dataset_type": "simple",
    "sliding_window": True,
    "max_seq_length": 1024,
	"mlm": False,
    "evaluate_during_training": True,
    "use_cached_eval_features": True,
    "evaluate_during_training_verbose": True,
    "save_optimizer_and_scheduler": False,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": True,
    "save_steps": -1,
    "train_batch_size":6,
    "num_train_epochs":3,
    "output_dir": f"{model_output_dir}/",
	"best_model_dir": f"{model_output_dir}/best_model"
}
with open(f"{model_output_dir}/training_parameters.json", 'w') as j:
	j.write(json.dumps(args, indent=4))

model = LanguageModelingModel("gpt2", model_output_dir, args=args)
model.train_model(train_file=train_file, eval_file=eval_file, args=args, verbose=True)

  0%|          | 0/1710 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/285 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/285 [00:00<?, ?it/s]

(7,
 {'global_step': [5, 7],
  'perplexity': [tensor(inf), tensor(6.4822e+30)],
  'eval_loss': [133.31558497094957, 70.94661173534695],
  'train_loss': [77.20818328857422, 67.88777923583984]})

In [105]:
from simpletransformers.language_generation import LanguageGenerationModel, LanguageGenerationArgs

args = {
		'max_length': 1024,
		'num_return_sequences': 1,
		'repetition_penalty': 1.01,
		'stop_token': '<|endoftext|>',
		'temperature': 0.8,
		'top_k': 80
}

text_model_generator = LanguageGenerationModel("gpt2",  f"{model_output_dir}/best_model", args=args, use_cuda=True)

question = "How do I Sign up?"

prompt = f'<|startoftext|><|startofquestion|>{question}<|endofquestion|><|startofreply|>'

result = "It don't work"
for text in text_model_generator.generate(prompt=prompt, args=args, verbose=True):
  result = text.replace(prompt, "").replace("<|endofreply|>", "")

pretty_response = f"<h1>Prompt: {question}</h1><br/><h1>Rely:{result}</h1>"

print(f"Prompt: {question}")
print(f"Reply: {result}")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│    9 │   │   'top_k': 80                                                                         │
│   10 }                                                                                           │
│   11                                                                                             │
│ ❱ 12 text_model_generator = LanguageGenerationModel("gpt2",  f"{model_output_dir}/best_model"    │
│   13                                                                                             │
│   14 question = "How do I Sign up?"                                                              │
│   15                                                                                             │
│                                                                                                  │
│ D:\workspaces\General\venv\lib\site-packages\simpletransformers\language_generation\language_gen │
│ eration_model.py:147 in __init__                                                                 │
│                                                                                                  │
│   144 │   │   │   **kwargs,                                                                      │
│   145 │   │   )                                                                                  │
│   146 │   │                                                                                      │
│ ❱ 147 │   │   self.model.to(self.device)                                                         │
│   148 │                                                                                          │
│   149 │   def generate(self, prompt=None, args=None, verbose=True):                              │
│   150                                                                                            │
│                                                                                                  │
│ D:\workspaces\General\venv\lib\site-packages\transformers\modeling_utils.py:1682 in to           │
│                                                                                                  │
│   1679 │   │   │   │   " model has already been set to the correct devices and casted to the co  │
│   1680 │   │   │   )                                                                             │
│   1681 │   │   else:                                                                             │
│ ❱ 1682 │   │   │   return super().to(*args, **kwargs)                                            │
│   1683 │                                                                                         │
│   1684 │   def half(self, *args):                                                                │
│   1685 │   │   # Checks if the model has been loaded in 8-bit                                    │
│                                                                                                  │
│ D:\workspaces\General\venv\lib\site-packages\torch\nn\modules\module.py:989 in to                │
│                                                                                                  │
│    986 │   │   │   │   │   │   │   non_blocking, memory_format=convert_to_format)                │
│    987 │   │   │   return t.to(device, dtype if t.is_floating_point() or t.is_complex() else No  │
│    988 │   │                                                                                     │
│ ❱  989 │   │   return self._apply(convert)                                                       │
│    990 │                                                                                         │
│    991 │   def register_backward_hook(                     